In [1]:
import pandas as pd
import numpy as np
import warnings
from matplotlib import pyplot as plt
from sklearn import metrics as m
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE  # doctest: +NORMALIZE_WHITESPACE
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from sklearn.metrics import classification_report
import xgboost as xgb
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from mlxtend.classifier import StackingClassifier
import itertools
from sklearn.pipeline import make_pipeline
from mlxtend.feature_selection import ColumnSelector
from mlxtend.classifier import StackingCVClassifier



warnings.filterwarnings("ignore")
randomseed = 7

Using TensorFlow backend.


1.Read the dataset and preprocess
======

In [2]:
x_original = pd.read_csv("../../XLable_onlyDiabeticRemoved.txt")

conditions = [
    (x_original.L100800 < 100),
    (x_original.L100800 >= 100) & (x_original.L100800 < 126),
    (x_original.L100800 >= 126),
]
choices = [0, 1, 2]
x_original["CLASS"] = np.select(conditions, choices, default=0)
x_original=x_original [['Unnamed: 0','L100800','L104600','L103000','S000300','L101700','L100700','FIELD_33',
                       'FIELD_38','FIELD_40','FIELD_31','SEX','AGE',#'CLASS',
                       'FIELD_16','FIELD_23','FIELD_15','FIELD_22','FIELD_17','FIELD_24'
                       ]]

In [3]:
y_original = pd.read_csv("../../TargetLable_onlyDiabeticRemoved.txt")

conditions = [
    (y_original.L100800 < 100),
    (y_original.L100800 >= 100) & (y_original.L100800 < 126),
    (y_original.L100800 >= 126),
]

choices = [0, 1, 2]
y_original["CLASS"] = np.select(conditions, choices, default=0)

y_original=y_original[['Unnamed: 0','CLASS' ]]

In [4]:
data = pd.merge(
    x_original, y_original, how="inner", left_on="Unnamed: 0", right_on="Unnamed: 0"
)

In [5]:
# filter the data set
data = data[data.FIELD_16 != 1]  # exclude people who are diagnosed for (diabetes)
data = data[data.FIELD_23 != 1]  # exclude people who are on medication for diabetes

data = data[
    data.FIELD_15 != 1
]  # exclude people who are diagnosed for (high blood pressure)
data = data[
    data.FIELD_22 != 1
]  # exclude people who are on medication for high blood pressure

data = data[data.FIELD_17 != 1]  # exclude people who are diagnosed for hyperlipidemia
data = data[
    data.FIELD_24 != 1
]  # exclude people who are on medication for hyperlipidemia


print(data.shape)

(169024, 20)


In [6]:
data=data[['L100800','L104600', 'L103000', 'S000300', 'L101700', 'L100700',
       'FIELD_33', 'FIELD_38', 'FIELD_40', 'FIELD_31', 'SEX', 'AGE','CLASS']]
data=data.dropna()
print(data.shape)
data.head()

(56542, 13)


,L100800,L104600,L103000,S000300,L101700,L100700,FIELD_33,FIELD_38,FIELD_40,FIELD_31,SEX,AGE,CLASS
2,78.0,5.28,41.0,20.2,15.0,3.8,1.0,2.0,1.0,0.0,1.0,46.0,0
5,90.0,5.74,50.0,25.5,12.0,3.4,1.0,0.0,1.0,0.0,1.0,52.0,0
10,86.0,5.83,45.0,21.2,17.0,3.9,1.0,0.0,1.0,1.0,1.0,37.0,0
11,86.0,4.73,54.0,22.0,30.0,4.2,1.0,2.0,3.0,0.0,1.0,39.0,0
20,87.0,5.60,340.0,24.6,26.0,4.7,1.0,0.0,2.0,0.0,1.0,59.0,0


2.Downsample the majority class and upsample the minority
====

In [7]:
diabetic = data[data.CLASS == 2]
prediabetic = data[data.CLASS == 1]
normal = data[data.CLASS == 0]

print(diabetic.shape[0], prediabetic.shape[0], normal.shape[0])

1045 17331 38166


In [8]:
diabetic_test = diabetic.sample(200, random_state=randomseed)
prediabetic_test = prediabetic.sample(200, random_state=randomseed)
normal_test = normal.sample(200, random_state=randomseed)
test = pd.concat([diabetic_test, prediabetic_test, normal_test])

diabetic_train = diabetic.drop(diabetic_test.index)
prediabetic_train = prediabetic.drop(prediabetic_test.index)
# .sample(
#     10 * diabetic_train.shape[0], random_state=randomseed
# )
normal_train = normal.drop(normal_test.index).sample(
    prediabetic_train.shape[0],random_state=randomseed
#     10 * diabetic_train.shape[0], random_state=randomseed
)
train = pd.concat([diabetic_train, diabetic_train, prediabetic_train, normal_train])

In [9]:

xtrain = train.iloc[:, :-1]
ytrain = train.iloc[:, -1]
xtest = test.iloc[:, :-1]
ytest = test.iloc[:, -1]



In [10]:
from sklearn.preprocessing import MinMaxScaler 
scaler = MinMaxScaler()
xtrain=scaler.fit_transform(xtrain)
xtest=scaler.transform(xtest)


In [11]:
from imblearn.over_sampling import SMOTE,SMOTENC  # doctest: +NORMALIZE_WHITESPACE
randomseed=42

sm = SMOTENC(random_state=randomseed,categorical_features=[6,7,8,9,10],sampling_strategy='minority')
X_res, y_res = sm.fit_resample(xtrain, ytrain)

print("Resampled dataset shape %s" % Counter(y_res))
print(
    y_res[y_res == 0].shape[0], y_res[y_res == 1].shape[0], y_res[y_res == 2].shape[0]
)
print(X_res.shape, y_res.shape)

xtrain = X_res
ytrain = y_res


Resampled dataset shape Counter({2: 17131, 1: 17131, 0: 17131})
17131 17131 17131
(51393, 12) (51393,)


3.Generate the classifier models based on the selected 12 features
====

In [12]:
# #rf
# rf_12 = RandomForestClassifier(
#     random_state=randomseed,
#     n_estimators=100, 
#     max_depth=12,
#     min_samples_split=3,
#     min_samples_leaf=10,
#     max_features="auto",
# )

In [13]:
# #xgboost
# xgb_model_12 = xgb.XGBClassifier(objective="binary:logistic", random_state=randomseed)

In [14]:
# #svm
# SVC_clf_12=SVC(C=70, cache_size=200, class_weight=None, coef0=0.0,
#     decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
#     kernel='linear', max_iter=-1, probability=False, random_state=None,
#     shrinking=True, tol=0.001, verbose=False)

In [15]:
# #lr
# lr_12=LogisticRegression(random_state=0)

In [16]:
# #knn
# kneigh_12 = KNeighborsClassifier(n_neighbors=150)

In [17]:
# #stacking classfier

# lr2_12 = LogisticRegression()
# temprf_12=RandomForestClassifier(n_estimators=500, random_state=42)
# temprf2_12=RandomForestClassifier()
# tempsvC_12= SVC()

# estimators_12 = [
#     rf_12,
#     xgb_model_12,
#     SVC_clf_12,
#     lr_12,
#     kneigh_12
# ]
# stackingc_12 = StackingClassifier(
#     classifiers=estimators_12,
#     meta_classifier=tempsvC_12,
#     verbose=2 
# )


In [18]:
# stackingc_12.fit(xtrain,ytrain)

In [19]:
# y_pred = stackingc_12.predict((xtest))

# print("accuracy \t", m.accuracy_score(ytest, y_pred))
# confmatrx = pd.DataFrame(m.confusion_matrix(ytest, y_pred))
# confmatrx.head()

4.Generate the classifier models based on the selected 5 features
===

In [20]:
# rf_5 = RandomForestClassifier(
#     random_state=randomseed,
#     n_estimators=100, 
#     max_depth=12,
#     min_samples_split=3,
#     min_samples_leaf=10,
#     max_features="auto",
# )

In [21]:
xgb_model_5 = xgb.XGBClassifier(objective="multi:softmax", random_state=randomseed)

In [22]:
# SVC_clf_5=SVC(C=70, cache_size=200, class_weight=None, coef0=0.0,
#     decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
#     kernel='linear', max_iter=-1, probability=False, random_state=None,
#     shrinking=True, tol=0.001, verbose=False)

In [23]:
# lr_5=LogisticRegression(random_state=0) 

In [24]:
# kneigh_5 = KNeighborsClassifier(n_neighbors=150) 

In [25]:
# lr2_5 = LogisticRegression()
# temprf_5=RandomForestClassifier(n_estimators=500, random_state=42)
# temprf2_5=RandomForestClassifier()
# tempsvC_5= SVC()

# estimators_5 = [
#     rf_5,
#     xgb_model_5,
#     SVC_clf_5,
#     lr_5,
#     kneigh_5
# ]
# stackingc_5 = StackingClassifier(
#     classifiers=estimators_5, 
#     meta_classifier=tempsvC_5,
#     verbose=2 
# )

In [26]:
# stackingc_5.fit(xtrain,ytrain)

In [27]:
# y_pred = stackingc_5.predict((xtest))

# print("accuracy \t", m.accuracy_score(ytest, y_pred))
# confmatrx = pd.DataFrame(m.confusion_matrix(ytest, y_pred))
# confmatrx.head()

In [28]:
# print(classification_report(ytest, y_pred))

In [29]:
# y_pred=stackingc_5.clfs_[2].predict((xtest))
# print("accuracy \t", m.accuracy_score(ytest, y_pred))
# confmatrx = pd.DataFrame(m.confusion_matrix(ytest, y_pred))
# confmatrx.head()

5.Generate multiple weak models using the existing features
====

In [30]:
from itertools import combinations
combcols=(list(combinations(np.arange(0,12,1), 10)))
len(combcols)

66

In [31]:
weakmodles=[]

In [32]:
for i in range(len(combcols)):
#     print(combcols[i])
    weakmodles.append(make_pipeline(ColumnSelector(cols=combcols[i]), xgb_model_5 ))

In [35]:
sclf1 = StackingCVClassifier(classifiers=weakmodles,verbose=2,
                           meta_classifier=LogisticRegression(),random_state=42)

In [36]:
sclf1.fit(xtrain,ytrain)

Fitting 66 classifiers...
Fitting classifier1: pipeline (1/66)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9),
                                drop_axis=False)),
                ('xgbclassifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.1,
                               max_delta_step=0, max_depth=3,
                               min_child_weight=1, missing=None,
                               n_estimators=100, n_jobs=1, nthread=None,
                               objective='multi:softmax', random_state=42,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               seed=None, silent=None, subsample=1,
                               verbosity=1))],
         verbose=False)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    6.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting classifier2: pipeline (2/66)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=(0, 1, 2, 3, 4, 5, 6, 7, 8, 10),
                                drop_axis=False)),
                ('xgbclassifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.1,
                               max_delta_step=0, max_depth=3,
                               min_child_weight=1, missing=None,
                               n_estimators=100, n_jobs=1, nthread=None,
                               objective='multi:softmax', random_state=42,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               seed=None, silent=None, subsample=1,
                               verbosity=1))],
         verbose=False)


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    6.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting classifier3: pipeline (3/66)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=(0, 1, 2, 3, 4, 5, 6, 7, 8, 11),
                                drop_axis=False)),
                ('xgbclassifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.1,
                               max_delta_step=0, max_depth=3,
                               min_child_weight=1, missing=None,
                               n_estimators=100, n_jobs=1, nthread=None,
                               objective='multi:softmax', random_state=42,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               seed=None, silent=None, subsample=1,
                               verbosity=1))],
         verbose=False)


...

Fitting classifier65: pipeline (65/66)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=(1, 3, 4, 5, 6, 7, 8, 9, 10, 11),
                                drop_axis=False)),
                ('xgbclassifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.1,
                               max_delta_step=0, max_depth=3,
                               min_child_weight=1, missing=None,
                               n_estimators=100, n_jobs=1, nthread=None,
                               objective='multi:softmax', random_state=42,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               seed=None, silent=None, subsample=1,
                               verbosity=1))],
         verbose=False)


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    6.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting classifier66: pipeline (66/66)
Pipeline(memory=None,
         steps=[('columnselector',
                 ColumnSelector(cols=(2, 3, 4, 5, 6, 7, 8, 9, 10, 11),
                                drop_axis=False)),
                ('xgbclassifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.1,
                               max_delta_step=0, max_depth=3,
                               min_child_weight=1, missing=None,
                               n_estimators=100, n_jobs=1, nthread=None,
                               objective='multi:softmax', random_state=42,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               seed=None, silent=None, subsample=1,
                               verbosity=1))],
         verbose=False)


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    6.4s finished


StackingCVClassifier(classifiers=[Pipeline(memory=None,
                                           steps=[('columnselector',
                                                   ColumnSelector(cols=(0, 1, 2,
                                                                        3, 4, 5,
                                                                        6, 7, 8,
                                                                        9),
                                                                  drop_axis=False)),
                                                  ('xgbclassifier',
                                                   XGBClassifier(base_score=0.5,
                                                                 booster='gbtree',
                                                                 colsample_bylevel=1,
                                                                 colsample_bynode=1,
                                                                 col

In [37]:
y_pred = sclf1.predict((xtest))

print("accuracy \t", m.accuracy_score(ytest, y_pred))
confmatrx = pd.DataFrame(m.confusion_matrix(ytest, y_pred))
confmatrx.head()

accuracy 	 0.72


,0,1,2
0,151,48,1
1,65,100,35
2,4,15,181


In [ ]:

from sklearn.model_selection import cross_val_score
scores = cross_val_score(sclf,
                         xtrain,ytrain,
                         cv=5,
                         scoring='accuracy')

In [69]:
len(sclf1.clfs_)

220

In [38]:
accuracy=[]
for i in range(len(sclf1.clfs_)):
    model=sclf1.clfs_[i]
    y_pred = model.predict((xtest))
    accuracy.append(m.accuracy_score(ytest, y_pred))
#     print(i)
    

In [39]:
accuracypd= pd.DataFrame()
accuracypd['accuracy']=accuracy
accuracypd['index']=np.arange(0,len(accuracy),1)
accuracypd['cols']=combcols
accuracypd.head()

,accuracy,index,cols
0,0.726667,0,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)"
1,0.730000,1,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 10)"
2,0.736667,2,"(0, 1, 2, 3, 4, 5, 6, 7, 8, 11)"
3,0.728333,3,"(0, 1, 2, 3, 4, 5, 6, 7, 9, 10)"
4,0.741667,4,"(0, 1, 2, 3, 4, 5, 6, 7, 9, 11)"


In [40]:
selectedclassifiers_info=accuracypd[accuracypd.accuracy>0.73].sort_values(by='accuracy',ascending=False).reset_index()
selectedclassifiers_info.head(10)

,level_0,accuracy,index,cols
0,9,0.745000,9,"(0, 1, 2, 3, 4, 5, 6, 9, 10, 11)"
1,42,0.745000,42,"(0, 1, 3, 4, 6, 7, 8, 9, 10, 11)"
2,5,0.743333,5,"(0, 1, 2, 3, 4, 5, 6, 7, 10, 11)"
3,8,0.743333,8,"(0, 1, 2, 3, 4, 5, 6, 8, 10, 11)"
4,25,0.743333,25,"(0, 1, 2, 3, 5, 6, 8, 9, 10, 11)"
5,40,0.741667,40,"(0, 1, 3, 4, 5, 6, 8, 9, 10, 11)"
6,39,0.741667,39,"(0, 1, 3, 4, 5, 6, 7, 9, 10, 11)"
7,14,0.741667,14,"(0, 1, 2, 3, 4, 5, 8, 9, 10, 11)"
8,4,0.741667,4,"(0, 1, 2, 3, 4, 5, 6, 7, 9, 11)"
9,18,0.740000,18,"(0, 1, 2, 3, 4, 6, 7, 9, 10, 11)"


In [41]:
selectedclassifiers_index=selectedclassifiers_info.index
selectedclassifiers_index

RangeIndex(start=0, stop=35, step=1)

In [42]:
print(list(selectedclassifiers_info.cols[:10]))

[(0, 1, 2, 3, 4, 5, 6, 9, 10, 11), (0, 1, 3, 4, 6, 7, 8, 9, 10, 11), (0, 1, 2, 3, 4, 5, 6, 7, 10, 11), (0, 1, 2, 3, 4, 5, 6, 8, 10, 11), (0, 1, 2, 3, 5, 6, 8, 9, 10, 11), (0, 1, 3, 4, 5, 6, 8, 9, 10, 11), (0, 1, 3, 4, 5, 6, 7, 9, 10, 11), (0, 1, 2, 3, 4, 5, 8, 9, 10, 11), (0, 1, 2, 3, 4, 5, 6, 7, 9, 11), (0, 1, 2, 3, 4, 6, 7, 9, 10, 11)]


6.Use only selected classifier from the previous model which have >73% accuracy
===

In [ ]:
weakmodles2=[]

In [ ]:
for i in range(len(selectedclassifiers_index)):
#     print(combcols[i])
    weakmodles2.append(make_pipeline(ColumnSelector(cols=selectedclassifiers_info.cols[i]), xgb_model_5 ))

In [ ]:
sclf2 = StackingCVClassifier(classifiers=weakmodles2,verbose=2,
                           meta_classifier=xgb.XGBClassifier(objective="multi:softmax", random_state=randomseed),
#                              LogisticRegression(),
                             random_state=42)

In [ ]:
sclf2.fit(xtrain,ytrain)

In [ ]:
y_pred = sclf2.predict((xtest))

print("accuracy \t", m.accuracy_score(ytest, y_pred))
confmatrx = pd.DataFrame(m.confusion_matrix(ytest, y_pred))
confmatrx.head()

7.Combine the 3 stacking classifiers
==============

In [ ]:
lr2_3 = LogisticRegression()
temprf_3=RandomForestClassifier(n_estimators=500, random_state=42)
temprf2_3=RandomForestClassifier()
tempsvC_3= SVC()

estimators_3 = [
    sclf2,  stackingc_5,stackingc_12    

]
stackingc_3 = StackingClassifier(
    classifiers=estimators_3, 
    meta_classifier=temprf2_3,
    verbose=2 
)

In [ ]:
stackingc_3.fit(xtrain,ytrain)

In [ ]:
y_pred = stackingc_3.predict((xtest))

print("accuracy \t", m.accuracy_score(ytest, y_pred))
confmatrx = pd.DataFrame(m.confusion_matrix(ytest, y_pred))
confmatrx.head()

In [ ]:

# from sklearn.pipeline import make_pipeline
# from mlxtend.feature_selection import ColumnSelector
# from mlxtend.classifier import StackingCVClassifier

# pipe1 = make_pipeline(rf)
# pipe11 = make_pipeline( xgb_model)
# pipe12 = make_pipeline( SVC_clf)
# pipe13 = make_pipeline( LogisticRegression())

# pipe2 = make_pipeline(ColumnSelector(cols=(0,1,2,9,10)), rf )
# pipe21 = make_pipeline(ColumnSelector(cols=(0,1,2,9,10)), xgb_model )
# pipe22 = make_pipeline(ColumnSelector(cols=(0,1,2,9,10)), SVC_clf )
# pipe22 = make_pipeline(ColumnSelector(cols=(0,1,2,9,10)), LogisticRegression() )



# sclf1 = StackingCVClassifier(classifiers=[    pipe1,    pipe11,    pipe12 ],
#                            meta_classifier=LogisticRegression(),random_state=42)

# sclf2 = StackingCVClassifier(classifiers=[    pipe2,    pipe21,    pipe22],
#                            meta_classifier=LogisticRegression(),random_state=42)

# sclf = StackingCVClassifier(classifiers=[
#    sclf1,sclf2
# ], 
#                             meta_classifier=LogisticRegression(),
#                             random_state=42)


